In [46]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import glob
import seaborn as sns  # for heatmaps
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import glob

# Pending

In [15]:
def import_sepctrograms(path,columns=columns):
    data = dict()
    for label in os.listdir(path):
        data[label] = dict()
        for f in listdir(path+'/'+label):
            user = f.split('.')[0].split('_')[1]
            df = pd.read_csv(path+'/'+label+'/'+f, names=columns)
            if user not in data[label]:
                data[label][user] = []
            data[label][user].append(df)
        print(label)
    return data

# path = "E:/external_data/Experiment2/spectrogram_data_by_activity_csv"
# data = import_sepctrograms(path)

In [9]:
columns = [f"col_{i+1}" for i in range(501)]
window_size=65
slide_size=30
dirc = 'E://external_data/Experiment4/Spectrogram_data_csv_files/CSI_data'

X,y  = import_data(dirc,columns=columns,window_size=window_size,slide_size=slide_size) # 

lay
picking
sit
walking
waving
stand_l
stand_s


In [ ]:


def cross_validation(item_func,X,y,n_splits):

    acc_score = []

    kf = KFold(n_splits)
    
    for train_index , test_index in kf.split(X):
        X_train , X_test = X[train_index], X[test_index]
        y_train , y_test = y[train_index], y[test_index]
        train_loader, test_loader = create_dataloader(X_train, X_test, y_train, y_test)
        
        # create model 
        model,optimizer,criterion = item_func()
        
        # train model
        model.fit(X_train,y_train)
        pred_values = model.predict(X_test)
        
        # evaluate
        acc = accuracy_score(pred_values , y_test)
        acc_score.append(acc)

    avg_acc_score = sum(acc_score)/k

    print('accuracy of each fold - {}'.format(acc_score))
    print('Avg accuracy : {}'.format(avg_acc_score))
    
    return 

In [56]:
def contrastive_loss(p1,p2,t=1):
    val = p1.dot(p2.transpose())/t
    val = np.exp(val)
    val = -np.log(np.diag(val)/np.sum(val,axis=1))
    return val

p1 = np.array([
 [ 0.70374682, -0.18682394, -0.68544673],
 [ 0.15465702,  0.32303224,  0.93366556],
 [ 0.53043332, -0.83523217, -0.14500935],
 [ 0.68285685, -0.73054075,  0.00409143],
 [ 0.76652431,  0.61500886,  0.18494479]])

p2 = p1 + 0.1*np.random.random(p1.shape)

# contrastive_loss(p1,p2,0.1)



In [86]:


def contrastive_loss(z1,z2,t=1):
    """
    One to N
    """
    a = torch.mm(z1,z2.transpose(0,1))
    b = torch.norm(z1,dim=1)*torch.norm(z2,dim=1)
    sim_mat = torch.div(a,b)
    mask = torch.ones_like(sim_mat,dtype=bool).fill_diagonal_(0)
    pos = torch.diag(sim_mat).reshape(-1,1)
    neg = sim_mat[mask].reshape(pos.shape[0],-1)
    logits = torch.cat((pos, neg), dim=1)
    labels = torch.zeros(pos.shape[0]).long()
    loss = torch.nn.CrossEntropyLoss()(logits,labels)
    return loss

z1 = torch.tensor(p1)
z2 = torch.tensor(p2)
z3 = torch.zeros_like(z1)

In [10]:
def intersect(d):
    d = np.array(d)
    ls = d[0]
    for new in d[1:]:
        ls = np.intersect1d(ls,new).tolist()
    return ls


def import_CsiPwr_data(directory):
    """
    import all spectrogram (in pair) in the directory
    """
    print("Importing Data ",end='')
    data = {'X':{},'y':{}}
    for label in os.listdir(directory):
        pfiles = []
        for modality in os.listdir(directory+'/'+label):
            pfiles.append([f.split('.')[0] for f in os.listdir(directory+'/'+label+'/'+modality)])

        print('>',end='')
        common_files = intersect(pfiles)
        
        
        for modality in os.listdir(directory+'/'+label):
            files = [directory+'/'+label+'/'+modality+'/'+pfilename+'.csv' for pfilename in common_files]
            X = import_spectrograms(files)
            y = np.full(X.shape[0], label)
            X[modality]
        
        # selcting available pairs
        pfiles_csi = [f.split('.')[0] for f in os.listdir(directory+'/'+label+'/'+'csi')]
        pfiles_pwr = [f.split('.')[0] for f in os.listdir(directory+'/'+label+'/'+'pwr')]
        available_pairs = np.intersect1d(pfiles_csi,pfiles_pwr).tolist()
        files_csi = [directory+'/'+label+'/'+'csi'+'/'+pfilename+'.csv' for pfilename in available_pairs]
        files_pwr = [directory+'/'+label+'/'+'pwr'+'/'+pfilename+'.csv' for pfilename in available_pairs]
        # importing
        X1 = import_spectrograms(files_csi)
        X2 = import_spectrograms(files_pwr)
        y = np.full(X1.shape[0], label)
        assert X1.shape[0] == X2.shape[0]
        data['X1'].append(X1)
        data['X2'].append(X2)
        data['y'].append(y)
    print(" Complete")
    return np.concatenate(data['X1']), np.concatenate(data['X2']), np.concatenate(data['y'])

### NUC pair

In [5]:
import numpy as np
import pandas as pd
import glob
import os
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import MinMaxScaler

In [206]:
def selections(*arg,**kwarg):
    size = int(arg[0].shape[0]*kwarg['p'])
    index = np.arange(0,arg[0].shape[0])
    test_selection = np.random.choice(index,size,replace=False)
    train_selection = np.array([i for i in index if i not in test_selection])
    return [i[train_selection] for i in arg],[i[test_selection] for i in arg]

In [223]:
def prepare_dataloader_pairdata(dirc,mode,p=None,resample=None):
    """
    Import pair data
    """
    if mode == 1:
        X1,X2,y = import_pair_data(dirc)
    elif mode == 2:
        X1,X2,y = import_CsiPwr_data(dirc)
    else:
        raise ValueError('Must be 1 or 2 for pair')
    
    X1 = X1.reshape(*X1.shape,1).transpose(0,3,1,2)
    X2 = X2.reshape(*X2.shape,1).transpose(0,3,1,2)
    y,lb = label_encode(y)
    
    ### train_test data
    training,validation = selections(X1,X2,y,p=0.2)
    X_test, y_test = validation[0],validation[2]
    
    ### train_test data
    if p:
        (X_train,y_train) , _ = selections(training[0],training[2],p=p)
    else:
        X_train,y_train = training[0],training[2]
    
    if resample:
        X_train,y_train,_ = resampling(X_train,y_train,y_train,oversampling=True)
        # X_test, y_test,_ = resampling(X_test, y_test,y_test,oversampling=False)
    
    ### Dataloader
    print('X1: ',training[0].shape,' X2: ',training[1].shape)
    print('X_train: ',X_train.shape,' y_train: ',y_train.shape,' X_test: ',X_test.shape,' y_test: ',y_test.shape)
    pretraindataset = TensorDataset(Tensor(training[0]),Tensor(training[1]))
    finetunedataset = TensorDataset(Tensor(X_train),Tensor(y_train).long())
    validatndataset = TensorDataset(Tensor(X_test), Tensor(y_test).long())
    pretrain_loader = DataLoader(pretraindataset, batch_size=bsz, shuffle=True, num_workers=num_workers, drop_last=True)
    finetune_loader = DataLoader(finetunedataset, batch_size=bsz, shuffle=True, num_workers=num_workers, drop_last=True)
    validatn_loader = DataLoader(validatndataset, batch_size=2000, shuffle=True, num_workers=num_workers)
    return pretrain_loader, finetune_loader, validatn_loader, lb

In [230]:
path = 'E:/external_data/Experiment4/Spectrogram_data_csv_files/CSI_data_pair'
pretrain_loader, finetune_loader, validatn_loader, lb = prepare_dataloader_pairdata(path,1)

Importing Data >>>>>> Complete
X1:  (576, 1, 65, 501)  X2:  (576, 1, 65, 501)
X_train:  (576, 1, 65, 501)  y_train:  (576,)  X_test:  (143, 1, 65, 501)  y_test:  (143,)


### LSTM

In [38]:
from models.baseline import LSTM
from models.cnn import create_vgg16,create_vgg16_atn
from models import Lambda
import torch
from torch import nn
from torch.nn import functional as F
from torchsummary import summary

In [3]:
class Attention(nn.Module):
    def __init__(self,feature_size):
        super(Attention, self).__init__()
        self.linear = nn.Linear(feature_size,1) 

    def forward(self,X):
        assert len(X.shape) == 3
        a = self.linear(X)
        a = torch.relu(a)
        a = F.softmax(a,dim=1)
        return a*X

class LSTM(nn.Module):
    def __init__(self,seq_size,feature_size,out_feature_size,attention=False):
        """
        2 layer LSTM model: feature_size --> 200 --> 3

        attr:
        seq_size: length of the sequence
        feature_size: feature size of each interval in the sequence
        """
        super(LSTM, self).__init__()
        self.lstm1 = nn.LSTM(feature_size,200)
        self.lstm2 = nn.LSTM(200,out_feature_size)
        if attention == True:
            self.attention = Attention(out_feature_size)
        else: 
            self.attention = Lambda(lambda x:x)


    def forward(self,X):
        X, _ = self.lstm1(X)
        X, _ = self.lstm2(X)
        X = self.attention(X)
        # X = torch.flatten(X,1)
        return X

In [26]:
seq = 33    
feature = 45
out = 7
mdl = LSTM(seq,feature,out,True)
attention = Attention(9)
sample = torch.rand(5,seq,feature)
attention = Attention(out)

In [43]:
out_shape = (3,1)
mdl = create_vgg16(out_shape)
sample = torch.rand(3,1,100,41)

In [44]:
summary(mdl,(1,100,41),64,device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
             Stack-1           [64, 3, 100, 41]               0
            Conv2d-2          [64, 64, 100, 41]           1,792
              ReLU-3          [64, 64, 100, 41]               0
            Conv2d-4          [64, 64, 100, 41]          36,928
              ReLU-5          [64, 64, 100, 41]               0
         MaxPool2d-6           [64, 64, 50, 20]               0
            Conv2d-7          [64, 128, 50, 20]          73,856
              ReLU-8          [64, 128, 50, 20]               0
            Conv2d-9          [64, 128, 50, 20]         147,584
             ReLU-10          [64, 128, 50, 20]               0
        MaxPool2d-11          [64, 128, 25, 10]               0
           Conv2d-12          [64, 256, 25, 10]         295,168
             ReLU-13          [64, 256, 25, 10]               0
           Conv2d-14          [64, 256,